#How To Build a Model: Step By Step


In [2]:
#Pytorch Imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.datasets import IMDB
from torchtext.transforms import ToTensor
from torchtext.vocab import build_vocab_from_iterator
#Pandas
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import time




In [3]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from transformers import AutoTokenizer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

c:\Users\kyley\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pick Your Universal Hyperparamaters:

In [4]:
batch_size = 1
epochs = 5


# Setup Your Model Workspace:

Your model architecture, and all of it's relevent code, will go in a folder at the location ./NNDL/(Your Model Name Here)


# Setup Your Dataset/Data Loaders Here

Pytorch usually takes a csv file. You need to write a function( For naming convention let's call it *create_torch_datasets()* ) that takes the universal dataset kyle got from parsing and seperates/loads it into data that you want your model to train on




In [13]:
#Create Custom Dataset Class:
class TextDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.root_dir = root_dir
        self.data = pd.read_csv(csv_file)
        self.mapping = {}
        self.cur_map = 1
        self.tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        text_o = os.path.join(self.root_dir,
            self.data.iloc[index, 0])
        text = self.tokenizer(text_o, padding="max_length")['input_ids']
        if len(text)!=512:
            print(text_o)
        category_text = os.path.join(self.root_dir,
            self.data.iloc[index, 1])
        if category_text in self.mapping:
            category = self.mapping[category_text]
        else:
            self.mapping[category_text] = self.cur_map
            category = self.cur_map
            self.cur_map+=1
        sample = {'text':text,'category':[category]}
        # print(f"self.mapping:{self.mapping}")
        return sample

In [14]:
#This is an example of what I'm talking about. The Fashion dataset is pretty easy bc it's function alr exists
# but you need to create your own function to make the training data and test_ data datasets
text_dataset = TextDataset(csv_file=r'C:\Users\kyley\Desktop\CS\C147\amazon-product-classification\data\products_noimg.csv',root_dir='/')
for i, sample in enumerate(text_dataset):
    print(i, len(sample['text']), len(sample['category']))
    if i==10:
        break
train_dataloader =  DataLoader(text_dataset, batch_size=batch_size,
                        shuffle=False, num_workers=0, drop_last=True)

# train_iter= IMDB(split="train")
# test_iter = IMDB(split="test")
# tokenizer = get_tokenizer("basic_english")
# def yield_tokens(data_iter):
#     for _, text in data_iter:
#         yield tokenizer(text)
# train_tokens = []
# vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
# vocab.set_default_index(vocab["<unk>"])
# test_tokens = []
# test_tokens = build_vocab_from_iterator(yield_tokens(test_iter), specials=["<unk>"])



0 512 1
1 512 1
2 512 1
3 512 1
4 512 1
5 512 1
6 512 1
7 512 1
8 512 1
9 512 1
10 512 1


Put the dataset into data loader, and check the shape, make sure it's how you want it.


In [12]:
# Create data loaders.
#train_dataloader = DataLoader(text_dataset, batch_size=batch_size,shuffle=True, num_workers=0)
#print(train_dataloader[0])
for i_batch, sample_batched in enumerate(train_dataloader):
    if len(sample_batched['text']) != 512:
        print(i_batch, len(sample_batched['text']),
          len(sample_batched['category']))

# Display image and label.
# train_features, train_labels = next(iter(train_dataloader))
# print(train_features)
# print(train_labels)
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

19 541 1
145 542 1
318 515 1
520 1083 1
708 570 1


KeyboardInterrupt: 

# Setup Model Architecture

1. Create your model architecture in your folder
2. Pick your loss function and optimizer

In [ ]:
from NNDL.TestModel.FCNN import NeuralNetwork 

model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


ModuleNotFoundError: No module named 'NNDL.TestModel.FCNN'

# Train the model
This should work to train your model. We may have to make some edits for different optimizations, but we can figure it out.


In [ ]:
from NNDL.Utils.solver import train,test
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295835  [   64/60000]
loss: 2.289212  [ 6464/60000]
loss: 2.277124  [12864/60000]
loss: 2.273642  [19264/60000]
loss: 2.242561  [25664/60000]
loss: 2.232709  [32064/60000]
loss: 2.223064  [38464/60000]
loss: 2.193398  [44864/60000]
loss: 2.187499  [51264/60000]
loss: 2.168628  [57664/60000]
Test Error: 
 Accuracy: 53.2%, Avg loss: 2.158337 

Epoch 2
-------------------------------
loss: 2.161626  [   64/60000]
loss: 2.152253  [ 6464/60000]
loss: 2.099909  [12864/60000]
loss: 2.119289  [19264/60000]
loss: 2.048779  [25664/60000]
loss: 2.004668  [32064/60000]
loss: 2.019566  [38464/60000]
loss: 1.940294  [44864/60000]
loss: 1.943159  [51264/60000]
loss: 1.875092  [57664/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.872346 

Epoch 3
-------------------------------
loss: 1.899469  [   64/60000]
loss: 1.867673  [ 6464/60000]
loss: 1.752991  [12864/60000]
loss: 1.803219  [19264/60000]
loss: 1.674740  [25664/60000]
loss: 1.634078  [32064/600